In [1]:
%matplotlib inline
import itertools
import os
os.environ['CUDA_VISIBLE_DEVICES']=""
import numpy as np
import gpflow
import gpflow.training.monitor as mon
import numbers
import matplotlib.pyplot as plt
import tensorflow as tf

# Demo: `gpflow.training.monitor`
In this notebook we'll demo how to use `gpflow.training.monitor` for logging the optimisation of a GPflow model.

## Creating the GPflow model
We first generate some random data and create a GPflow model.

Under the hood, GPflow gives a unique name to each model which is used to name the Variables it creates in the TensorFlow graph containing a random identifier. This is useful in interactive sessions, where people may create a few models, to prevent variables with the same name conflicting. However, when loading the model, we need to make sure that the names of all the variables are exactly the same as in the checkpoint. This is why we pass name="SVGP" to the model constructor, and why we use gpflow.defer_build().

In [2]:
np.random.seed(0)
X = np.random.rand(10000, 1) * 10
Y = np.sin(X) + np.random.randn(*X.shape)
Xt = np.random.rand(10000, 1) * 10
Yt = np.sin(Xt) + np.random.randn(*Xt.shape)

with gpflow.defer_build():
    m = gpflow.models.SVGP(X, Y, gpflow.kernels.RBF(1), gpflow.likelihoods.Gaussian(),
                           Z=np.linspace(0, 10, 5)[:, None],
                           minibatch_size=100, name="SVGP")
    m.likelihood.variance = 0.01
m.compile()

Let's compute log likelihood before the optimisation

In [3]:
print('LML before the optimisation: %f' % m.compute_log_likelihood())

LML before the optimisation: -1271605.621944


We will be using a TensorFlow optimiser. All TensorFlow optimisers have a support for `global_step` variable. Its purpose is to track how many optimisation steps have occurred. It is useful to keep this in a TensorFlow variable as this allows it to be restored together with all the parameters of the model.

The code below creates this variable using a monitor's helper function. It is important to create it before building the monitor in case the monitor includes a checkpoint task. This is because the checkpoint internally uses the TensorFlow Saver which creates a list of variables to save. Therefore all variables expected to be saved by the checkpoint task should exist by the time the task is created.

In [4]:
session = m.enquire_session()
global_step = mon.create_global_step(session)

## Construct the monitor

Next we need to construct the monitor. `gpflow.training.monitor` provides classes that are building blocks for the monitor. Essengially, a monitor is a function that is provided as a callback to an optimiser. It consists of a number of tasks that may be executed at each step, subject to their running condition.

In this example, we want to:
- log certain scalar parameters in TensorBoard,
- log the full optimisation objective (log marginal likelihood bound) periodically, even though we optimise with minibatches,
- store a backup of the optimisation process periodically,
- log performance for a test set periodically.

We will define these tasks as follows:

In [5]:
print_task = mon.PrintTimingsTask().with_name('print')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

sleep_task = mon.SleepTask(0.01).with_name('sleep').with_name('sleep')

saver_task = mon.CheckpointTask('./monitor-saves').with_name('saver')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

file_writer = mon.LogdirWriter('./model-tensorboard')

model_tboard_task = mon.ModelToTensorBoardTask(file_writer, m).with_name('model_tboard')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

lml_tboard_task = mon.LmlToTensorBoardTask(file_writer, m).with_name('lml_tboard')\
    .with_condition(mon.PeriodicIterationCondition(100))\
    .with_exit_condition(True)

As the above code shows, each task can be assigned a name and running conditions. The name will be shown in the task timing summary.

There are two different types of running conditions: `with_condition` controls execution of the task at each iteration in the optimisation loop. `with_exit_condition` is a simple boolean flag indicating that the task should also run at the end of optimisation.
In this example we want to run our tasks periodically, at every iteration or every 10th or 100th iteration.

Notice that the two TensorBoard tasks will write events into the same file. It is possible to share a file writer between multiple tasks. However it is not possible to share the same event location between multiple file writers. An attempt to open two writers with the same location will result in error.


## Custom tasks
We may also want to perfom certain tasks that do not have pre-defined `Task` classes. For example, we may want to compute the performance on a test set. Here we create such a class by extending `BaseTensorBoardTask` to log the testing benchmarks in addition to all the scalar parameters.

In [6]:
class CustomTensorBoardTask(mon.BaseTensorBoardTask):
    def __init__(self, file_writer, model, Xt, Yt):
        super().__init__(file_writer, model)
        self.Xt = Xt
        self.Yt = Yt
        self._full_test_err = tf.placeholder(gpflow.settings.tf_float, shape=())
        self._full_test_nlpp = tf.placeholder(gpflow.settings.tf_float, shape=())
        self._summary = tf.summary.merge([tf.summary.scalar("test_rmse", self._full_test_err),
                                         tf.summary.scalar("test_nlpp", self._full_test_nlpp)])
    
    def run(self, context: mon.MonitorContext, *args, **kwargs) -> None:
        minibatch_size = 100
        preds = np.vstack([self.model.predict_y(Xt[mb * minibatch_size:(mb + 1) * minibatch_size, :])[0]
                            for mb in range(-(-len(Xt) // minibatch_size))])
        test_err = np.mean((Yt - preds) ** 2.0)**0.5
        self._eval_summary(context, {self._full_test_err: test_err, self._full_test_nlpp: 0.0})

        
custom_tboard_task = CustomTensorBoardTask(file_writer, m, Xt, Yt).with_name('custom_tboard')\
    .with_condition(mon.PeriodicIterationCondition(100))\
    .with_exit_condition(True)

Now we can put all these tasks into a monitor.

In [7]:
monitor_tasks = [print_task, model_tboard_task, lml_tboard_task, custom_tboard_task, saver_task, sleep_task]
monitor = mon.Monitor(monitor_tasks, session, global_step)

## Running the optimisation
We finally get to running the optimisation.

We may want to continue a previously run optimisation by resotring the TensorFlow graph from the latest checkpoint. Otherwise skip this step.

In [8]:
if os.path.isdir('./monitor-saves'):
    mon.restore_session(session, './monitor-saves')

In [9]:
optimiser = gpflow.train.AdamOptimizer(0.01)

with mon.Monitor(monitor_tasks, session, global_step, print_summary=True) as monitor:
    optimiser.minimize(m, step_callback=monitor, maxiter=450, global_step=global_step)

file_writer.close()

Iteration 10	total itr.rate 12.98/s	recent itr.rate 12.98/s	opt.step 10	total opt.rate 14.72/s	recent opt.rate 14.72/s
Iteration 20	total itr.rate 19.32/s	recent itr.rate 37.77/s	opt.step 20	total opt.rate 28.96/s	recent opt.rate 887.41/s
Iteration 30	total itr.rate 24.44/s	recent itr.rate 51.97/s	opt.step 30	total opt.rate 42.54/s	recent opt.rate 690.68/s
Iteration 40	total itr.rate 27.92/s	recent itr.rate 48.73/s	opt.step 40	total opt.rate 55.70/s	recent opt.rate 771.83/s
Iteration 50	total itr.rate 30.70/s	recent itr.rate 51.09/s	opt.step 50	total opt.rate 68.73/s	recent opt.rate 1068.65/s
Iteration 60	total itr.rate 33.03/s	recent itr.rate 53.21/s	opt.step 60	total opt.rate 80.79/s	recent opt.rate 658.99/s
Iteration 70	total itr.rate 34.66/s	recent itr.rate 49.25/s	opt.step 70	total opt.rate 92.57/s	recent opt.rate 741.85/s
Iteration 80	total itr.rate 35.12/s	recent itr.rate 38.67/s	opt.step 80	total opt.rate 103.80/s	recent opt.rate 687.96/s
Iteration 90	total itr.rate 36.37/s	rec

 27%|██▋       | 27/100 [00:00<00:00, 268.51it/s]

Iteration 100	total itr.rate 37.33/s	recent itr.rate 48.98/s	opt.step 100	total opt.rate 125.23/s	recent opt.rate 743.73/s


100%|██████████| 100/100 [00:00<00:00, 391.92it/s]


Iteration 110	total itr.rate 31.69/s	recent itr.rate 12.62/s	opt.step 110	total opt.rate 135.53/s	recent opt.rate 760.34/s
Iteration 120	total itr.rate 32.47/s	recent itr.rate 44.38/s	opt.step 120	total opt.rate 144.96/s	recent opt.rate 618.62/s
Iteration 130	total itr.rate 33.50/s	recent itr.rate 54.19/s	opt.step 130	total opt.rate 155.35/s	recent opt.rate 1114.54/s
Iteration 140	total itr.rate 34.41/s	recent itr.rate 53.25/s	opt.step 140	total opt.rate 165.32/s	recent opt.rate 995.97/s
Iteration 150	total itr.rate 35.22/s	recent itr.rate 52.35/s	opt.step 150	total opt.rate 174.51/s	recent opt.rate 787.66/s
Iteration 160	total itr.rate 35.73/s	recent itr.rate 45.79/s	opt.step 160	total opt.rate 183.04/s	recent opt.rate 684.73/s
Iteration 170	total itr.rate 36.26/s	recent itr.rate 47.44/s	opt.step 170	total opt.rate 191.39/s	recent opt.rate 708.79/s
Iteration 180	total itr.rate 36.82/s	recent itr.rate 50.08/s	opt.step 180	total opt.rate 199.34/s	recent opt.rate 677.99/s
Iteration 190	t

...

Iteration 410	total itr.rate 37.50/s	recent itr.rate 15.92/s	opt.step 410	total opt.rate 334.72/s	recent opt.rate 721.82/s
Iteration 420	total itr.rate 37.69/s	recent itr.rate 47.42/s	opt.step 420	total opt.rate 339.00/s	recent opt.rate 711.56/s
Iteration 430	total itr.rate 37.90/s	recent itr.rate 49.11/s	opt.step 430	total opt.rate 343.45/s	recent opt.rate 765.62/s
Iteration 440	total itr.rate 38.08/s	recent itr.rate 48.17/s	opt.step 440	total opt.rate 347.65/s	recent opt.rate 734.16/s
Iteration 450	total itr.rate 38.26/s	recent itr.rate 48.47/s	opt.step 450	total opt.rate 351.91/s	recent opt.rate 763.46/s


 49%|████▉     | 49/100 [00:00<00:00, 482.62it/s]

Iteration 450	total itr.rate 37.49/s	recent itr.rate 0.00/s	opt.step 450	total opt.rate 317.80/s	recent opt.rate 0.00/s


100%|██████████| 100/100 [00:00<00:00, 483.37it/s]


Tasks execution time summary:
print:	0.0373 (sec)
model_tboard:	0.1832 (sec)
lml_tboard:	1.2192 (sec)
custom_tboard:	1.1748 (sec)
saver:	3.8945 (sec)
sleep:	4.5394 (sec)


Now lets compute the log likelihood again. Hopefully we will see an increase in its value

In [10]:
print('LML after the optimisation: %f' % m.compute_log_likelihood())

LML after the optimisation: -68705.124191
